In [1]:
from solver import Solver

The demo example for running inference with signal family solvers:

In [ ]:
import numpy as np
import scipy.sparse as sp
X = np.random.randn(20, 50)
y = np.random.randn(20)
A = sp.csr_matrix(np.triu((np.random.rand(50,50) < 0.05).astype(int), k=1))
A = A + A.T
L = sp.csgraph.laplacian(A, normed=False)

# models = ["Proximal", "Lasso", "GFL_Matlab", "Lasso_Sklearn", "Adaptive_Grace"]
models = ["signal_family"]
""" 
Here c is the upper bound the method introduces on the 2-norm of the weights
When they have the ground_truth weights, they take the value of the 2-norm of the ground_truth weights
For inference case, we don't know the ground_truth weights, so we set c to 1 by default
"""
solver = Solver(models, c=1)
res = solver.inference(X, y, L, A, k=2)


Running model signal_family
0 6.244805447420771 0.07119870330196261
1 5.457618055258544 0.11109818755379323
2 5.092851337782441 0.1535242922374815
3 4.773497545744348 0.16240480557835882
4 4.518316015407045 0.16926038809275684
5 4.305108268895647 0.17517929437059634
6 3.9129085210977257 0.21104387673086034
7 3.7167308685170597 0.22422040293321951
8 3.5187134716376596 0.25025090427509733
9 3.337537036207682 0.253496981737139
10 3.1845307480016407 0.26360746709068206
11 2.9555686587616323 0.2840928842394121
12 2.8270318169154454 0.2964985770028855
13 2.704931729226556 0.31628387330706087
14 2.5994774785211736 0.32168670661667426
15 2.4421650136490776 0.34033374017965273
16 2.3509898482601312 0.351667810925453
17 2.2665233733781807 0.36793991279166033
18 2.1961909067239573 0.36878007325752094
19 2.123187518984561 0.3745896747831168
20 2.0603981900650434 0.3753546276510691
21 1.9509052879051862 0.39155086951613016
22 1.89454557826041 0.39359399811934653
23 1.8382590532262182 0.394240908474

In [3]:
res

defaultdict(list,
            {'gen-mp': array([ 0.        , -0.16941714,  0.        ,  0.        ,  0.08963141,
                     0.        , -0.31777121,  0.        ,  0.        ,  0.        ,
                     0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
                     0.        , -0.15004545,  0.        ,  0.        ,  0.1046631 ,
                    -0.09839376,  0.        ,  0.        ,  0.        ,  0.        ,
                     0.        ,  0.01839167,  0.        ,  0.        ,  0.        ,
                     0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
                     0.06527192, -0.04968098,  0.05782613,  0.        ,  0.        ,
                    -0.05943596,  0.        ,  0.        ,  0.        ,  0.        ,
                     0.        ,  0.        ,  0.        ,  0.16368526, -0.1771172 ]),
             'dmo-acc-fw': array([ 0.        , -0.10509804,  0.        ,  0.        ,  0.06039216,
                     